# Use jupyter to test 

In [1]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text)

    return HTML(html)
hide_toggle()

In [2]:
x = 1
y = 2
print('Result is {} + {}'.format(x, y))

hide_toggle()

Result is 1 + 2


In [ ]:
# hide_toggle(for_next=True)

In [3]:
hide_toggle(for_next=True)

In [5]:
import requests as r
from ipywidgets import Label, BoundedFloatText, BoundedIntText, Dropdown, Button, Output, VBox,widgets
prescribe_label = Label('Spam Detection ml models')
age_text = widgets.Text(
    value='',
    placeholder='Paste your  description here!',
    description='String:',
    disabled=False
)
prescribe_button = Button(description="Presribe")
prescribe_output = Output()

# Button click event handlers ...
def prescribe_button_on_click(b):
    item ={
      "message": 
         age_text.value
    }
    prediction = r.post("http://192.168.99.100:8000/predict-spam",  json=item)
    recommended_spam = prediction.json()
    prescribe_output.clear_output()
    with prescribe_output:

        print(f"The analyse of message is {recommended_spam}")
prescribe_button.on_click(prescribe_button_on_click)
vbox_prescribe = VBox([prescribe_label, age_text,prescribe_button, prescribe_output ])
vbox_prescribe

# Streamlit : 


In [15]:
%%writefile streamlit_spam.py
import streamlit as st
import requests as r
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import re 

st.title('Spam Detection  App')
st.write('*Note: it will take up to 30 seconds to run the app.*')
form = st.form(key='message-form')
user_input = form.text_area('Enter your text')
submit = form.form_submit_button('Submit')

#load model data
url = "C:/Users/rzouga/Desktop/ALLINHERE/ALLINHERE/Deploy_Spam_Detection/models/spam_classifier.joblib"
#f = "C:/Users/rzouga/Desktop/ALLINHERE/ALLINHERE/FraudDetection/DeployPipeComplet/models/pipeline_model_lgbm_final.joblib"
# download model from Dropbox, cache it and load the model into the app
@st.cache(allow_output_mutation=True)
def load_model(url):
    model = joblib.load(url)
    return model   
# Preprocess Heleper 
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # Effectively removes HTML markup tags
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

if submit:
    fig, ax = plt.subplots()
    model = load_model(url)
    message = preprocessor(user_input)
    label = model.predict([message])[0]
    score = model.predict_proba([message])[0][1]
    if label == 'ham':
        st.success(f'This is a {label} : (score: {score})')
    else:
        st.error(f'OOPS it is a {label} : (score: {score})')
        
    classes = {0:'ham',1:'spam'}
    class_labels = list(classes.values())

    st.write("The predicted class is ",label)
    prob_ham= 1-score
    prob_spam = score
    probs = [prob_ham,prob_spam]
    ax = sns.barplot(probs ,class_labels, palette="winter", orient='h')
    ax.set_yticklabels(class_labels,rotation=0)
    plt.title("Probabilities of the Data belonging to each class")
    for index, value in enumerate(probs):
        plt.text(value, index,str(value))
    st.pyplot(fig) 

Overwriting streamlit_spam.py


In [13]:
!streamlit run streamlit_spam.py  

^C


In [17]:
import streamlit as st
import requests as r
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import re 

In [18]:
import session_info
session_info.show()